Zunächst die Standardimports der Libraries

In [35]:
import numpy as np
import cv2
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.feature import local_binary_pattern

Import der Daten
(erstmal nur die ersten 5 Bilder) (mittlerweile alle)

In [36]:
currentPath = Path().cwd()

targetPath = currentPath.parent / 'data' / 'KGT_noDefect_simplified'
imsNoF = [[str(file), False] for file in targetPath.iterdir() if file.is_file()]

targetPath = currentPath.parent / 'data' / 'KGT_pitting_simplified'
imsF = [[str(file), True] for file in targetPath.iterdir() if file.is_file()]

ims = imsNoF + imsF

del(imsNoF, imsF, currentPath, targetPath)

zur veranschaulichung Umwandlung in Grey Scale
<h1> Überschrift </h1>

Für den Moment damit bereits dreifache Trainingsdaten. Kanten halte ich für äußerst hilfreich.

In [41]:
def processImage(path, size=(150,150)):
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, size)

    # Feature 1: einfache Grayscale
    grayFlat = img.flatten()

    # Feature 2: sobel
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    sobel = cv2.magnitude(sobelx, sobely)
    sobelFlat = sobel.flatten()

    # Feature 3: Histogramm
    hist = cv2.calcHist([img], [0], None, [16], [0,256])
    hist = cv2.normalize(hist,hist).flatten()

    # Feature 4: LBP (Local Binary Pattern)
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp, bins = np.arange(0,10), range= (0,9), density = True)

    features = np.concatenate([grayFlat,sobelFlat,hist,lbp_hist])
    return features

In [42]:
for i in range(len(ims)):
    processedImage = processImage(ims[i][0])
    if processedImage is not None:
        ims[i][0] = processedImage

In [43]:
images, labels = zip(*ims)
X = np.array(images)
y = np.array(labels)
XTrain, XTest, yTrain, yTest = train_test_split(X,y, stratify=y, test_size=0.3, random_state=42)

In [44]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(XTrain, yTrain)
yPred = clf.predict(XTest)
print(classification_report(yTest,yPred))

              precision    recall  f1-score   support

       False       0.96      0.90      0.93       194
        True       0.93      0.97      0.95       255

    accuracy                           0.94       449
   macro avg       0.94      0.94      0.94       449
weighted avg       0.94      0.94      0.94       449

